In [1]:
import os

In [2]:
from extractDataSrc.Mdi import Mdi
from extractDataSrc.Lasco import Lasco
from extractDataSrc.InSitu import InSitu
from extractDataSrc.Eit195 import Eit195

/Users/majirky/Desktop/Vigil-ML/vigil-dev/vigil-ml/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# check if destination folders for images are present
destination_folders = ["data_processed/eit", "data_processed/mdi/mag", "data_processed/mdi/con" ,"data_processed/lasco/c2", "data_processed/lasco/c3", "data_processed/in_situ"]

for folder in destination_folders:
    os.makedirs(folder, exist_ok=True)

In [4]:
helper_folders = ["data_csv", "data_fits/eit"]

for folder in destination_folders:
    os.makedirs(folder, exist_ok=True)

In [6]:
start_datetime = "2001-03-21 00:00:00"
stop_datetime = "2001-04-10 23:59:59"

### Extracting SOHO/MDI magnetograms and continuum

In [6]:
mdi = Mdi(start_datetime, stop_datetime)
# mdi.extract_data("mag")
# works! :)

In [7]:
#mdi.extract_data("con")

### Extracting SOHO/Lasco c2 and c3 images

In [8]:
lasco = Lasco(start_datetime, stop_datetime)
# works! :)

In [9]:
lasco.extract_data("c2")

1311 / 1311
success


In [10]:
lasco.extract_data("c3")

859 / 859
success


### Extracting SOHO/V_p, N_p and WIND/B_z time-series data

In [8]:
insitu = InSitu(start_datetime, stop_datetime)
insitu.extract_data("V_p", make_png=True)
insitu.extract_data("N_p", make_png=True)
insitu.extract_data("B_z", make_png=True)
# works! :)

### Extracting SOHO/EIT 195 images via VSO links to images

In [7]:
eit195 = Eit195(start_datetime, stop_datetime)
eit195.extract_data("data_csv/eit_event1.csv", quality_check=True)

transforming and asving image... 2298
